# Assignment 2

In [1]:
# All libraries required
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

2024-05-10 01:45:24.769801: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 01:45:25.162392: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 01:45:27.619836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Tensorflow Version
print(tf.__version__)

2.16.1


In [3]:
# Allocate more memory - https://www.tensorflow.org/guide/gpu

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2024-05-10 01:45:31.508346: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-10 01:45:32.810886: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-10 01:45:32.810953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-10 01:45:32.843943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-10 01:45:32.844099: I external/local_xla/xla/stream_executor

In [80]:
# Get data - https://www.tensorflow.org/tutorials/load_data/images#load_data_using_a_keras_utility

# Parameters
batch_size = 125912
IMG_SIZE = (512, 512)
data_dir = './hagridset'
seed_no = 388741
validation_split = 0.3

# Dataset - https://stackoverflow.com/questions/66036271/splitting-a-tensorflow-dataset-into-training-test-and-validation-sets-from-ker

# Auto split to training set (70%)
training_data_set = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                       image_size = IMG_SIZE,
                                                       validation_split = validation_split,
                                                       subset = 'training',
                                                       seed = seed_no,
                                                       color_mode = 'grayscale',
                                                       shuffle = True)

# Auto split to validation set and test set (30%)
validation_data_set = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                       image_size = IMG_SIZE,
                                                       validation_split = validation_split,
                                                       subset = 'validation',
                                                       seed = seed_no,
                                                       color_mode = 'grayscale',
                                                       shuffle = True)

# Split further to validation set (10%) and test set (20%)
validation_batches = tf.data.experimental.cardinality(validation_data_set)
test_data_set = validation_data_set.take((2 * validation_batches) // 3)
validation_data_set = validation_data_set.skip((2 * validation_batches) // 3)

KeyboardInterrupt: 

In [5]:
# Number of classes
num_classes = len(training_data_set.class_names)
num_classes

18

In [6]:
# Efficiency for data loading
AUTOTUNE = tf.data.AUTOTUNE

#train_data_set = training_data_set.cache().prefetch(buffer_size=AUTOTUNE)
#validation_data_set = validation_data_set.cache().prefetch(buffer_size=AUTOTUNE)
#test_data_set = test_data_set.cache().prefetch(buffer_size=AUTOTUNE)

## Model 1: Grayscale

In [7]:
from tensorflow.keras import layers

# Model building
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 1)))
model.add(layers.Conv2D(12, 3, strides=2, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(18, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(24, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(48, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(96, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(192, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(192, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 12)   │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 12)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 18)   │         1,962 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 18)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 24)     │         3,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 32)     │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 48)     │        13,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 96)       │        41,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 4, 4, 128)      │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 2, 2, 192)      │       221,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 1, 1, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 192)            │        37,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 18)             │         3,474 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 441,004 (1.68 MB)

 Trainable params: 441,004 (1.68 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [65]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [64]:
history = model.fit(training_data_set, validation_data=validation_data_set, epochs=100, callbacks=[callback])

Epoch 1/100


KeyboardInterrupt: 

In [12]:
model.evaluate(test_data_set)

787/787 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.8200 - loss: 0.6272


[0.6337441205978394, 0.8193297386169434]

## Model 2: Colour

In [81]:
# Get data - https://www.tensorflow.org/tutorials/load_data/images#load_data_using_a_keras_utility

# Parameters
batch_size = 125912
IMG_SIZE = (512, 512)
data_dir = './hagridset'
seed_no = 388741
validation_split = 0.3

# Dataset - https://stackoverflow.com/questions/66036271/splitting-a-tensorflow-dataset-into-training-test-and-validation-sets-from-ker

# Auto split to training set (70%)
training_data_set = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                       image_size = IMG_SIZE,
                                                       validation_split = validation_split,
                                                       subset = 'training',
                                                       seed = seed_no,
                                                       color_mode = "rgb",
                                                       label_mode='categorical',
                                                       shuffle = True)

# Auto split to validation set and test set (30%)
validation_data_set = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                       image_size = IMG_SIZE,
                                                       validation_split = validation_split,
                                                       subset = 'validation',
                                                       seed = seed_no,
                                                       color_mode = "rgb",
                                                       label_mode='categorical',
                                                       shuffle = True)

# Split further to validation set (10%) and test set (20%)
validation_batches = tf.data.experimental.cardinality(validation_data_set)
test_data_set = validation_data_set.take((2 * validation_batches) // 3)
validation_data_set = validation_data_set.skip((2 * validation_batches) // 3)

Found 125912 files belonging to 18 classes.
Using 88139 files for training.
Found 125912 files belonging to 18 classes.
Using 37773 files for validation.


In [82]:
# Model building
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model2.add(layers.Conv2D(12, 3, strides=2, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(18, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(24, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(48, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(96, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(192, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Flatten())
model2.add(layers.Dense(192, activation='relu'))
model2.add(layers.Dense(num_classes, activation='softmax'))

In [83]:
model2.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_86 (Conv2D)              │ (None, 256, 256, 12)   │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_86 (MaxPooling2D) │ (None, 128, 128, 12)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_87 (Conv2D)              │ (None, 128, 128, 18)   │         1,962 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_87 (MaxPooling2D) │ (None, 64, 64, 18)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_88 (Conv2D)              │ (None, 64, 64, 24)     │         3,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_88 (MaxPooling2D) │ (None, 32, 32, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_89 (Conv2D)              │ (None, 32, 32, 32)     │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_89 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_90 (Conv2D)              │ (None, 16, 16, 48)     │        13,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_90 (MaxPooling2D) │ (None, 8, 8, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_91 (Conv2D)              │ (None, 8, 8, 96)       │        41,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_91 (MaxPooling2D) │ (None, 4, 4, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_92 (Conv2D)              │ (None, 4, 4, 128)      │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_92 (MaxPooling2D) │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_93 (Conv2D)              │ (None, 2, 2, 192)      │       221,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_93 (MaxPooling2D) │ (None, 1, 1, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 192)            │        37,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 18)             │         3,474 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 441,220 (1.68 MB)

 Trainable params: 441,220 (1.68 MB)

 Non-trainable params: 0 (0.00 B)

In [84]:
model2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [85]:
history2 = model2.fit(training_data_set, validation_data=validation_data_set, epochs=100, callbacks=[callback])

Epoch 1/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 178s 63ms/step - accuracy: 0.2216 - loss: 2.4292 - val_accuracy: 0.6629 - val_loss: 1.0541
Epoch 2/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 176s 64ms/step - accuracy: 0.7128 - loss: 0.8987 - val_accuracy: 0.7726 - val_loss: 0.7471
Epoch 3/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 199s 72ms/step - accuracy: 0.7875 - loss: 0.6738 - val_accuracy: 0.8094 - val_loss: 0.6611
Epoch 4/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 206s 75ms/step - accuracy: 0.8182 - loss: 0.5804 - val_accuracy: 0.8125 - val_loss: 0.6171
Epoch 5/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.8382 - loss: 0.5182

2024-05-10 03:31:40.194245: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134220032 bytes after encountering the first element of size 134220032 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-05-10 03:32:16.640616: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134220032 bytes after encountering the first element of size 134220032 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


2755/2755 ━━━━━━━━━━━━━━━━━━━━ 241s 87ms/step - accuracy: 0.8382 - loss: 0.5182 - val_accuracy: 0.8216 - val_loss: 0.6083
Epoch 6/100
   3/2755 ━━━━━━━━━━━━━━━━━━━━ 2:39 58ms/step - accuracy: 0.8194 - loss: 0.6211

2024-05-10 03:32:32.718639: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134220032 bytes after encountering the first element of size 134220032 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


2754/2755 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8492 - loss: 0.4764

...

2755/2755 ━━━━━━━━━━━━━━━━━━━━ 155s 56ms/step - accuracy: 0.8908 - loss: 0.3566 - val_accuracy: 0.8400 - val_loss: 0.5855
Epoch 15/100
   3/2755 ━━━━━━━━━━━━━━━━━━━━ 2:50 62ms/step - accuracy: 0.8924 - loss: 0.3479

2024-05-10 03:56:00.111717: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134220032 bytes after encountering the first element of size 134220032 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


2755/2755 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.8890 - loss: 0.3610

2024-05-10 03:57:57.680724: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134220032 bytes after encountering the first element of size 134220032 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-05-10 03:58:22.162973: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134220032 bytes after encountering the first element of size 134220032 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


2755/2755 ━━━━━━━━━━━━━━━━━━━━ 156s 56ms/step - accuracy: 0.8890 - loss: 0.3610 - val_accuracy: 0.8530 - val_loss: 0.5532


In [86]:
model2.evaluate(test_data_set)

  1/787 ━━━━━━━━━━━━━━━━━━━━ 4:43 361ms/step - accuracy: 0.9375 - loss: 0.1831

2024-05-10 04:01:49.468688: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134220032 bytes after encountering the first element of size 134220032 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-05-10 04:01:49.468898: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134220032 bytes after encountering the first element of size 134220032 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


787/787 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.8407 - loss: 0.5375


[0.5307518839836121, 0.843869149684906]

# Model 3: Transfer Learning

In [93]:
model3_base = tf.keras.applications.vgg16(
    weights='imagenet',
    input_shape=(IMG_SIZE[0], IMG_SIZE[1],3),
    include_top=False)
model3_base.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1],3))
x = model3_base(inputs, training=False)
outputs = tf.keras.layers.Dense(num_classes)(x)

model3 = tf.keras.Model(inputs, outputs)

TypeError: 'module' object is not callable

In [ ]:
model3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model3.summary()

In [ ]:
history3 = model3.fit(training_data_set, validation_data=validation_data_set, epochs=100, callbacks=[callback])